# Structured Output

`irouter` supports structured output out of the box. Simply pass a schema to the `response_format` parameter in a call to `Call` or `Chat`. This notebook will explain how to define schemas and get structured output from `irouter`.

In [1]:
import json
from irouter import Call

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Call or Chat class
from dotenv import load_dotenv

load_dotenv();

## Response Schema

`response_format` is a parameter which expects a dictionary with the following structure. Below shows an annotated example of a response format schema.

For details on the schema format, check the [OpenRouter docs](https://openrouter.ai/docs/features/structured-outputs#using-structured-outputs).

In [2]:
response_format = {
    # Define the desired type, which is almost always json_schema
    "type": "json_schema",
    # Define the schema
    "json_schema": {
        # Name for your schema (not that important what you call this)
        "name": "dogs",
        # Strict mode ensures that the response is exactly as defined in the schema
        "strict": True,
        # Here you define the actual dictionary output you would like to receive
        "schema": {
            "type": "object",
            # Properties are the keys of the object you would like returned
            # For each property, you can define the type (number, string, etc.) and a short description
            "properties": {
                # 1. Amount of dogs in the image
                "amount": {"type": "number", "description": "Amount of dogs"},
                # 2. Breed of the dogs
                "breed": {"type": "string", "description": "Breed of the dogs"},
                # 3. Background of the image
                "background": {
                    "type": "string",
                    "description": "Background of the image",
                },
            },
            # Required defines the keys that are always present in the response
            "required": ["amount", "breed", "background"],
            # Setting additionalProperties to False ensures that no other keys
            # besides the ones defined in required are present in the response.
            "additionalProperties": False,
        },
    },
}

## LLM Call

Not all models support structured outputs. Check the [model overview](https://openrouter.ai/models?fmt=cards&supported_parameters=response_format) to see which models support `response_format`. 

In this example we identify the `amount` of dogs in an image, their `breed` and the `background` of the image. As you can see there are 7 German Shepherds in the image. 

We expect the response to be something like:

```json
{
    "amount": 7,
    "breed": "German Shepherd",
    "background": "outdoor yard"
}
```

![](https://i.pinimg.com/564x/6e/7a/87/6e7a87269e40e053e3fa33e7f0e3eef8.jpg)

I found `gpt-5-mini` to be a good choice for relatively easy structured outputs with the option for multi-modal input. 

Below, GPT 5 Mini is called with the image and our schema. The output is then parsed into a dictionary with `json.loads`.


In [3]:
c = Call("openai/gpt-5-mini")
r = c(
    ["https://i.pinimg.com/564x/6e/7a/87/6e7a87269e40e053e3fa33e7f0e3eef8.jpg"],
    response_format=response_format,
)
j = json.loads(r)
j

{'amount': 7,
 'breed': 'German Shepherd',
 'background': 'outdoor grassy yard with trees and a wooden fence'}

In [4]:
j["amount"]

7

In [5]:
print(f"The image shows {j['amount']} {j['breed']}s in a {j['background']}.")

The image shows 7 German Shepherds in a outdoor grassy yard with trees and a wooden fence.
